In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import re

In [125]:

DEFAULT_PATH_DATA_CREDIT = r'E:\Hinevics\Dropbox\Programm\GitHub\Credit-Train\Data\credit_train.csv'

data = pd.read_csv(filepath_or_buffer=DEFAULT_PATH_DATA_CREDIT, encoding='utf-8', sep=';', index_col='client_id')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data preparation

In [126]:
# rename open_account_flg
data.rename(columns = {'open_account_flg,,': 'open_account_flg'}, inplace=True)

In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 1 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  object 
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   education             170741 non-null  object 
 9   living_region         170554 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  object 
dtypes: float64(5), int64(1), object(8)
memory usage: 19.

In [128]:
# Сколько тут nan параметров
data.isna().sum()

gender                     0
age                        3
marital_status             3
job_position               0
credit_sum                 2
credit_month               0
tariff_id                  0
score_shk                  7
education                  5
living_region            192
monthly_income             5
credit_count            9230
overdue_credit_count    9230
open_account_flg           0
dtype: int64

In [129]:
# overdue_credit_count

# Я решил удалить все аккануты у которых неизветсно количество кредитов.
# поскльку отнести их в какую-то другую категорию не могу, тк количество это не категориальный параметр.
data.drop(index=data[data['overdue_credit_count'].isna()].index, inplace=True)

In [130]:
data.isna().sum()

gender                    0
age                       3
marital_status            3
job_position              0
credit_sum                2
credit_month              0
tariff_id                 0
score_shk                 7
education                 5
living_region           174
monthly_income            5
credit_count              0
overdue_credit_count      0
open_account_flg          0
dtype: int64

In [132]:
# living_region
index_nan_living_region  = data.living_region.isna().index
# data.living_region[data['living_region'].isna()] = 'NON'

In [133]:
data['living_region'].fillna(value="NON", inplace=True)
data['living_region'].isna().sum()

In [ ]:
data['living_region'].unique()

In [ ]:
stop_words = ['АО', 'КРАЙ', 'РЕСПУБЛИКА', 'Г', 'АОБЛ', 'ОКРУГ', 'ЭЛ', 'ОБЛ', 'РЕСП']
def del_words_regions(arg):
    arg = re.sub(r"[\/\/\\\.,(0)\-']", ' ', str(arg))
    words = re.split(' ', arg)
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return ' '.join(new_words)
del_words_regions('ОБЛ САРАТОВСКАЯ')

In [ ]:
regions = data['living_region'].unique()
regions = pd.Series(data=regions, name='regions', index=regions)


In [ ]:
list_uniqie_words_from_regions = {re.sub(pattern=r"[\/\/\\\.,(0)\-']",
                                         repl='',
                                         string=j) for www in map(lambda x: x.split(' ') if type(x) is not float else [], regions.values) for j in www}

In [ ]:
regions = regions.map(del_words_regions)

In [ ]:
# вносим финальные корректировки в regions
regions['ЧУКОТСКИЙ АO'] = 'ЧУКОТСКИЙ'
regions['ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['РЕСП ЧУВАШСКАЯ - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['РЕСПУБЛИКАТАТАРСТАН'] = 'ТАТАРСТАН'
regions['ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'] = 'МОСКОВСКАЯ'
regions['ПЕРМСКАЯ ОБЛ'] = 'ПЕРМСКИЙ'
regions['ОРЁЛ'] = 'ОРЛОВСКАЯ'
regions['Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['МЫТИЩИНСКИЙ Р-Н'] = 'МОСКОВСКАЯ'
regions['МОСКОВСКИЙ П'] = 'МОСКОВСКАЯ'
regions['КАМЧАТСКАЯ ОБЛАСТЬ'] = 'КАМЧАТСКИЙ'
regions['ДАЛЬНИЙ ВОСТОК'] = 'МОСКОВСКАЯ'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н'] = 'ВЛАДИМИРСКАЯ'
regions['ГОРЬКОВСКАЯ ОБЛ'] = 'НИЖЕГОРОДСКАЯ'
regions['ЭВЕНКИЙСКИЙ АО'] = 'КРАСНОЯРСКИЙ'
regions['ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['СЕВ. ОСЕТИЯ - АЛАНИЯ'] = 'СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ'
regions['РЕСП. САХА (ЯКУТИЯ)'] ='САХА/ЯКУТИЯ/'
regions['РЕСПУБЛИКА САХА'] = 'САХА/ЯКУТИЯ/'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['САХА'] = 'САХА/ЯКУТИЯ/'
regions['98'] = 'САНКТ-ПЕТЕРБУРГ'
regions['74'] = 'ЧЕЛЯБИНСКАЯ'
regions['РОССИЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['Г. ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['БРЯНСКИЙ'] = 'БРЯНСКАЯ'

In [ ]:
data['living_region'] = data['living_region'].map(regions)

In [ ]:
data.isna().sum()

In [ ]:
# # score_shk
data.drop(index=data[data['score_shk'].isna()].index, inplace=True)

In [ ]:
data[data['score_shk'] == 'NON']

In [ ]:
data['score_shk'] = data['score_shk'].map(lambda x: str(x).replace(',', '.')).astype('float')

# Classification by open_account_flg

## Purpose: classification of users by opening a credit account